In [1]:
!pip install smbprotocol

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 6.4 MB/s eta 0:00:00


In [4]:
!ping upcourtinenas

/bin/bash: line 1: ping: command not found


In [3]:
import uuid
from smbprotocol.connection import Connection
from smbprotocol.session import Session
from smbprotocol.tree import TreeConnect
from smbprotocol.open import Open

# Connection details
server = "upcourtinenas"
share_name = "UPPERLIMB"
username = "maetz"

from getpass import getpass
password = getpass("Enter SMB password: ")

# Establish an SMB connection
connection = Connection(uuid.uuid4(), server, 445)
connection.connect()

# Create a session with authentication
session = Session(connection, username, password)
session.connect()

# Connect to the share on the server
tree = TreeConnect(session, r"\\{}\\{}".format(server, share_name))
tree.connect()

Enter SMB password: ··········


ValueError: Failed to connect to 'upcourtinenas:445': [Errno -2] Name or service not known

In [ ]:
import numpy as np
import os
from tqdm import tqdm as tqdm

from pathlib import Path

import sys
sys.path.append('/Users/theomaetz/Downloads/MA3/INL/Setup/lighthouse-main')

from lighthouse.data_loader.load_chuv import load_absd
from lighthouse import metadata

In [ ]:
os.environ["DATA_DIR"] = "/Volumes"
X = []
Dir = []
Train_Dir = []
testing_sessions = metadata.get_testing_sessions()

In [ ]:
len(testing_sessions)

8

In [ ]:
import numpy as np
from tqdm import tqdm
from pathlib import Path

# Function to read checkpoint from file
def read_checkpoint(filename='checkpoint_test.txt'):
    try:
        with open(filename, 'r') as f:
            return int(f.read().strip())
    except FileNotFoundError:
        return 0  # Start from the beginning if no checkpoint exists

# Function to write checkpoint to file
def write_checkpoint(session_index, filename='checkpoint_test.txt'):
    with open(filename, 'w') as f:
        f.write(str(session_index + 1))

In [ ]:


# Load previous checkpoints if available
try:
    X = list(np.load('Test_data.npy', allow_pickle=True))
    Dir = list(np.load('Ch_test.npy', allow_pickle=True))
    Train_Dir = list(np.load('dir_test.npy', allow_pickle=True))
    start_session = read_checkpoint()  # Resume from the last saved session
    print("Found File")
    print(f"Starting from session: {start_session}")
except FileNotFoundError:
    X, Dir, Train_Dir = [], [], []
    start_session = 0  # Start from the beginning if no previous data


try:
    for i, train_ses in enumerate(tqdm(testing_sessions[start_session:]), start=start_session):
        print(f"Current session: {train_ses, i}")
        absd_dirs = [
            p.resolve() for p in Path(train_ses).glob("**") if p.is_dir() and ("ABSD" in p.name or "absd" in p.name)
        ]

        for absd_dir in absd_dirs:
            dt5_files = sorted(list(Path(absd_dir).glob("*.dt5")))

            if len(dt5_files) == 0:
                continue

            n_files = len(dt5_files)
            counter = 1
            skip_this = False
            for file in dt5_files:
                if not int(file.stem[-2:]) == n_files:
                    skip_this = True
                    continue
                if not counter == int(file.stem[-6:-4]):
                    skip_this = True
                    continue
                counter += 1

            if skip_this:
                continue

            # Process signals
            signals, ch_names = load_absd(absd_dir, return_is_updating=True, return_states=True)
            update_index = int(np.argwhere(np.array(ch_names) == 'is_updating'))
            update = np.argwhere(signals[update_index, :] == 1)
            print(f"Signals shape: {signals.shape}")

            new_sig = np.squeeze(signals[:, update])
            print(f"New signal shape: {new_sig.shape}")
            X.append(new_sig)
            Dir.append(ch_names)
            Train_Dir.append(absd_dir)

            np_array_X = np.empty(len(X), dtype=object)
            for j, x in enumerate(X):
                np_array_X[j] = x
            print(f"length of data: {len(X)}")

        # Save after every checkpoint interval

        np.save('Ch_test.npy', np.array(Dir, dtype=object), allow_pickle=True)
        np.save('Test_data.npy', np.array(np_array_X, dtype=object), allow_pickle=True)
        np.save('dir_test.npy', np.array(Train_Dir, dtype=object), allow_pickle=True)
        write_checkpoint(i)
        print(f"Checkpoint saved after session {i}")

except KeyboardInterrupt:
    print(f"Script interrupted at session {i}. Saving current progress...")
    write_checkpoint(i)
    print("Progress saved after interruption.")

except Exception as e:
    print(f"An error occurred: {e}")
    print(f"Saving progress at session {i} before exiting...")
    write_checkpoint(i)
    raise  # Re-raise the exception after saving

else:
    # Final save after the loop completes successfully
    np_array_X = np.empty(len(X), dtype=object)
    for j, x in enumerate(X):
        np_array_X[j] = x
    print(f"length of data_: {len(X)}")

    np.save('Ch_test.npy', np.array(Dir, dtype=object), allow_pickle=True)
    np.save('Test_data.npy', np.array(np_array_X, dtype=object), allow_pickle=True)
    np.save('dir_test.npy', np.array(Train_Dir, dtype=object), allow_pickle=True)
    write_checkpoint(i + 1)
    print("Final save complete.")


Found File
Starting from session: 5


  0%|          | 0/3 [00:00<?, ?it/s]

Current session: (PosixPath('/Volumes/__UP2/0_RAW_DATA/UP2_001/UP2001_2024_01_10_Rehabilitation_day24'), 5)


2024-10-17 08:56:11.638 | INFO     | lighthouse.data_loader.load_chuv:concatenate_dt5:497 - Loading /Volumes/__UP2/0_RAW_DATA/UP2_001/UP2001_2024_01_10_Rehabilitation_day24/wisci/ABSD data/2024_01_10_13_21_24_01of02.dt5
2024-10-17 09:01:22.193 | INFO     | lighthouse.data_loader.load_chuv:load_dt5:196 - Scenario Name: classBCIViewerAvatarAllDegreesCourtine
2024-10-17 09:01:22.210 | INFO     | lighthouse.data_loader.load_chuv:load_dt5:199 - Sub Scenario Name: classBSISubScenarioUpperJointsClassif
2024-10-17 09:01:27.954 | INFO     | lighthouse.data_loader.load_chuv:concatenate_dt5:497 - Loading /Volumes/__UP2/0_RAW_DATA/UP2_001/UP2001_2024_01_10_Rehabilitation_day24/wisci/ABSD data/2024_01_10_13_21_24_02of02.dt5
/var/folders/bv/m44_t4vj2sl8r_5ljvsxts2w0000gn/T/ipykernel_68521/2724180887.py:44: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. 

Signals shape: (88, 296888)
New signal shape: (88, 0)
length of data: 15


2024-10-17 09:04:29.876 | INFO     | lighthouse.data_loader.load_chuv:concatenate_dt5:497 - Loading /Volumes/__UP2/0_RAW_DATA/UP2_001/UP2001_2024_01_10_Rehabilitation_day24/wisci/2024_01_10_13_16_38 (rec2)/ABSD data/2024_01_10_13_31_30_01of09.dt5
2024-10-17 09:10:15.859 | INFO     | lighthouse.data_loader.load_chuv:load_dt5:196 - Scenario Name: classBCIViewerAvatarAllDegreesCourtine
2024-10-17 09:10:15.865 | INFO     | lighthouse.data_loader.load_chuv:load_dt5:199 - Sub Scenario Name: classBSISubScenarioUpperJointsClassif
2024-10-17 09:10:21.199 | INFO     | lighthouse.data_loader.load_chuv:concatenate_dt5:497 - Loading /Volumes/__UP2/0_RAW_DATA/UP2_001/UP2001_2024_01_10_Rehabilitation_day24/wisci/2024_01_10_13_16_38 (rec2)/ABSD data/2024_01_10_13_31_30_02of09.dt5
2024-10-17 09:14:29.742 | INFO     | lighthouse.data_loader.load_chuv:concatenate_dt5:497 - Loading /Volumes/__UP2/0_RAW_DATA/UP2_001/UP2001_2024_01_10_Rehabilitation_day24/wisci/2024_01_10_13_16_38 (rec2)/ABSD data/2024_01_1

Signals shape: (88, 1583914)
New signal shape: (88, 300723)
length of data: 16


2024-10-17 09:53:18.449 | INFO     | lighthouse.data_loader.load_chuv:concatenate_dt5:497 - Loading /Volumes/__UP2/0_RAW_DATA/UP2_001/UP2001_2024_01_10_Rehabilitation_day24/wisci/2024_01_10_14_17_46 (rec3)/ABSD data/2024_01_10_14_21_11_01of18.dt5
2024-10-17 10:00:54.195 | INFO     | lighthouse.data_loader.load_chuv:load_dt5:196 - Scenario Name: classBCIViewerAvatarAllDegreesCourtine
2024-10-17 10:00:54.209 | INFO     | lighthouse.data_loader.load_chuv:load_dt5:199 - Sub Scenario Name: classBSISubScenarioUpperJointsClassif
2024-10-17 10:00:59.730 | INFO     | lighthouse.data_loader.load_chuv:concatenate_dt5:497 - Loading /Volumes/__UP2/0_RAW_DATA/UP2_001/UP2001_2024_01_10_Rehabilitation_day24/wisci/2024_01_10_14_17_46 (rec3)/ABSD data/2024_01_10_14_21_11_02of18.dt5
2024-10-17 10:06:00.868 | INFO     | lighthouse.data_loader.load_chuv:concatenate_dt5:497 - Loading /Volumes/__UP2/0_RAW_DATA/UP2_001/UP2001_2024_01_10_Rehabilitation_day24/wisci/2024_01_10_14_17_46 (rec3)/ABSD data/2024_01_1